In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
from urllib import request
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
url = "https://www.jumia.co.ke/all-products/"
requests = requests.get(url)
## Parsing the html soup
soup = BeautifulSoup(requests.text,"html.parser")
items = soup.find_all("article", attrs = {"class":"prd _fb col c-prd"})

In [3]:
print(len(items))

40


In [4]:
print(items[0])

<article class="prd _fb col c-prd"><a class="btn _i _rnd -mas -fsh0 -me-start _wslt _sec" data-pop-open="addToWishlist" data-pop-trig="atw" data-sku="GA297ST2BTNHJNAFAMZ" href="/customer/account/login/?tkWl=GA297ST2BTNHJNAFAMZ-158186998&amp;return=%2Fall-products%2F" rel="nofollow"><svg aria-label="Add to wishlist" class="ic -f-or5" height="16" viewbox="0 0 24 24" width="16"><use xlink:href="https://www.jumia.co.ke/assets_he/images/i-icons.a66628fd.svg#saved-items"></use></svg></a><a class="core" data-gtm-brand="Garnier" data-gtm-category="Health &amp; Beauty/Beauty &amp; Personal Care/Personal Care/Skin Care/Face/Treatments &amp; Masks/Serums" data-gtm-dimension23="5662" data-gtm-dimension26="1502" data-gtm-dimension27="4.3" data-gtm-dimension28="1" data-gtm-dimension37="0" data-gtm-dimension43="BDEAL|BF2018|BLF|BLF_02|Boost|COL_260|COL_268|COL_269|COL_272|COL_275|COL_276|COL_278|COL_317|CP_24|CP_26|CP_MT123|CP_UN104|CP_UN105|CP_UN132|CP_UN34|DOW_10|DOW_100|JMALL|TBOOST|TOP" data-gtm-

In [5]:
first_items = items[0]
first_items_name =first_items.find("h3",attrs = {"class":"name"}).text
print(first_items_name)

Garnier Anti-Blemish Charcoal Serum With AHA + BHA & Niacinamide For Acne Prone Skin


In [6]:
first_items_price = first_items.find("div", attrs = {"class":"prc"}).text
first_items_price

'KSh 1,400'

In [7]:
first_items = items[0]
first_items_link = first_items.find("a", attrs = {"class":"core"}).get("href")
print(first_items_link)

/garnier-anti-blemish-charcoal-serum-with-aha-bha-niacinamide-for-acne-prone-skin-119519704.html


In [8]:
##discount
first_items = items[0]
first_items_discount = first_items.find("div", attrs = {"class":"bdg _dsct _sm"}).text
print(first_items_discount)

20%


In [9]:
records= []
for item in items:
    try:
        item_name =item.find("h3",attrs = {"class":"name"}).text
        current_price =item.find("div", attrs = {"class":"prc"}).text
        old_price_element = item.find("div", attrs = {"class":"old"})
        item_discount_element = item.find("div", attrs = {"class":"bdg _dsct _sm"})
        item_link = item.find("a", attrs = {"class":"core"}).get("href")

        if old_price_element: 
            old_price = old_price_element.text
        else:
            old_price = None
        if item_discount_element:
            item_discount = item_discount_element.text
        else:
            item_dicount = None
        records.append(( item_name, current_price,old_price, item_discount, item_link))

    except AttributeError:
        records.append(None)
print(len(records))

records

40


[('Garnier Anti-Blemish Charcoal Serum With AHA + BHA & Niacinamide For Acne Prone Skin',
  'KSh 1,400',
  'KSh 1,750',
  '20%',
  '/garnier-anti-blemish-charcoal-serum-with-aha-bha-niacinamide-for-acne-prone-skin-119519704.html'),
 ('Garnier Even & Matte  Vitamin C Booster Serum 15ml',
  'KSh 850',
  None,
  '20%',
  '/garnier-even-matte-vitamin-c-booster-serum-15ml-115443314.html'),
 ('Garnier Pure Active 3 In1 Charcoal Anti Blackhead Mask Wash Scrub -150ml',
  'KSh 1,036',
  'KSh 1,295',
  '20%',
  '/garnier-pure-active-3-in1-charcoal-anti-blackhead-mask-wash-scrub-150ml-15855244.html'),
 ('Garnier Even & Matte  Vitamin C Booster Serum 30ml.',
  'KSh 1,400',
  'KSh 1,750',
  '20%',
  '/garnier-even-matte-vitamin-c-booster-serum-30ml.-47090194.html'),
 ('Garnier Bye Acne & Dark Spots Kit.',
  'KSh 3,608',
  'KSh 4,245',
  '15%',
  '/garnier-bye-acne-dark-spots-kit.-74339805.html'),
 ('Garnier Brightening Night Serum With 10% Pure Vitamin C',
  'KSh 2,495',
  None,
  '15%',
  '/garnie

In [10]:
import time
import requests
from bs4 import BeautifulSoup

base_url = "https://www.jumia.co.ke/all-products/?page={}#catalog-listing"
records = []

for page_number in range(1, 51):
    url = base_url.format(page_number)
    time.sleep(5)
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        print(f"Page {page_number} scrapped successfully")
        items = soup.find_all("article", attrs={"class": "prd _fb col c-prd"})
        for item in items:
            try:
                item_name = item.find("h3", attrs={"class": "name"}).text
                current_price = item.find("div", attrs={"class": "prc"}).text
                old_price_element = item.find("div", attrs={"class": "old"})
                item_discount_element = item.find("div", attrs={"class": "bdg _dsct _sm"})
                item_link = item.find("a", attrs={"class": "core"}).get("href")
                
                if old_price_element:
                    old_price = old_price_element.text
                else:
                    old_price = None
                if item_discount_element:
                    item_discount = item_discount_element.text
                else:
                    item_discount = None
                records.append((item_name, current_price, old_price, item_discount, item_link))
            except AttributeError:
                records.append(None)
    else:
        print(f"Page {page_number} not scrapped")

Page 1 scrapped successfully
Page 2 scrapped successfully
Page 3 scrapped successfully
Page 4 scrapped successfully
Page 5 scrapped successfully
Page 6 scrapped successfully
Page 7 scrapped successfully
Page 8 scrapped successfully
Page 9 scrapped successfully
Page 10 scrapped successfully
Page 11 scrapped successfully
Page 12 scrapped successfully
Page 13 scrapped successfully
Page 14 scrapped successfully
Page 15 scrapped successfully
Page 16 scrapped successfully
Page 17 scrapped successfully
Page 18 scrapped successfully
Page 19 scrapped successfully
Page 20 scrapped successfully
Page 21 scrapped successfully
Page 22 scrapped successfully
Page 23 scrapped successfully
Page 24 scrapped successfully
Page 25 scrapped successfully
Page 26 scrapped successfully
Page 27 scrapped successfully
Page 28 scrapped successfully
Page 29 scrapped successfully
Page 30 scrapped successfully
Page 31 scrapped successfully
Page 32 scrapped successfully
Page 33 scrapped successfully
Page 34 scrapped su

In [11]:
# Save records to CSV file
import csv
csv_file = "jumia_products.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Item Name", "Current Price", "Old Price", "Discount", "Item Link"])
    writer.writerows(records)

print("Data saved to", csv_file)

Data saved to jumia_products.csv


In [12]:
df = pd.read_csv("jumia_products.csv")

In [13]:
print("First few rows of the DataFrame:")
df.head(5)

First few rows of the DataFrame:


,Item Name,Current Price,Old Price,Discount,Item Link
0,Garnier Anti-Blemish Charcoal Serum With AHA +...,"KSh 1,400","KSh 1,750",20%,/garnier-anti-blemish-charcoal-serum-with-aha-...
1,Garnier Even & Matte Vitamin C Booster Serum ...,KSh 850,NaN,NaN,/garnier-even-matte-vitamin-c-booster-serum-15...
2,Garnier Pure Active 3 In1 Charcoal Anti Blackh...,"KSh 1,036","KSh 1,295",20%,/garnier-pure-active-3-in1-charcoal-anti-black...
3,Garnier Even & Matte Vitamin C Booster Serum ...,"KSh 1,400","KSh 1,750",20%,/garnier-even-matte-vitamin-c-booster-serum-30...
4,Garnier Bye Acne & Dark Spots Kit.,"KSh 3,608","KSh 4,245",15%,/garnier-bye-acne-dark-spots-kit.-74339805.html


In [14]:
print("\nSummary statistics:")
df.describe().T


Summary statistics:


,count,unique,top,freq
Item Name,2000,1904,Fashion Men's Hawaiian Ink Print Bottom-down ...,3
Current Price,2000,1264,KSh 899,21
Old Price,1742,1049,"KSh 1,000",38
Discount,1742,60,50%,250
Item Link,2000,1923,/nordic-humidifier-generic-mpg433225.html,2


In [15]:
print("\nData types of columns:")
df.dtypes


Data types of columns:


Item Name        object
Current Price    object
Old Price        object
Discount         object
Item Link        object
dtype: object

In [16]:
print("\nMissing values:")
df.isnull().sum()


Missing values:


Item Name          0
Current Price      0
Old Price        258
Discount         258
Item Link          0
dtype: int64